In [32]:
import pandas as pd, numpy as np, re
import csv
from datetime import datetime

# Doing some clean up

In [33]:
# Doing some clean up
def remove_quotes(label):
    return re.sub(r"'", "", label)

def read_remove_quotes(filename, outputname, row_length ):
    with open(filename) as f:
        df = csv.reader(f)
        table_f = []
        index = 0
        for line in df:
            if ((len(line) == row_length) & (len(line)>0)):
                table_f.append([word for word in line])
                index += 1

    with open(outputname, 'wb') as csvfile:
        write_f = csv.writer(csvfile, delimiter=',')
        for row in table_f: write_f.writerow(row)

In [34]:


def read_csv(filename):
    with open(filename) as f:
        df = csv.reader(f)
        w = [line for line in df]
        return w

In [35]:
cuisine = read_csv("Cuisine.txt")
read_remove_quotes("Cuisine.txt", "Cuisine_clean.csv", 2)

In [36]:
webex = read_csv("WebExtract.txt")
read_remove_quotes("WebExtract.txt", "WebExtract_clean.csv", 15)

In [37]:
action = read_csv("Action.txt")
laction = [len(a) for a in action] # each row has length 4
read_remove_quotes("Action.txt", "Action_clean.csv", 4)

In [38]:
webextract = pd.io.parsers.read_csv('WebExtract.txt')
webextract.head()

/opt/conda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462,7188924968,8,2014-03-03 00:00:00,D,10F,2,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000
1,30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225,7182875005,39,2014-07-01 00:00:00,F,06A,23,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,10B,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
3,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234,7184443838,27,2014-05-29 00:00:00,D,08C,10,A,2014-05-29 00:00:00,2014-09-04 06:01:28.403000000
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,02G,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000


In [39]:
# what datatypes should be
web_type = {"CAMIS" : str, "DBA" : str,"BORO"  :str, "BUILDING" : str, "STREET" : str,
"ZIPCODE" : str, "PHONE" : str,"CUISINECODE" : str,
"INSPDATE" : datetime, "ACTION" : str,"VIOLCODE" : str,
"SCORE" : int, "CURRENTGRADE" : str, "GRADEDATE" : datetime,
"RECORDDATE" : datetime}

# for (key, value) in web_type.iteritems():
#    if web[key].dtype == float: web[key] = web[key].astype(int)
#    web[key] = web[key].astype(value)

In [40]:
#Return a list of tuples of the form
# (zipcode, mean grade, standard error, number of inspections)`
# for each of the 183 zipcodes in the city with over 100 inspections.  
# Sort the list in ascending order by score.

zips = webextract[["ZIPCODE", "SCORE"]]
zips = zips.dropna()

In [41]:
zips["ZIPCODE"] = zips["ZIPCODE"].astype(int).astype(str)
zips["SCORE"] = zips["SCORE"].astype(np.int)

In [42]:
# add sum, mean, stddev per zipcodes
gzips = zips.groupby("ZIPCODE")

gzips_agg = gzips["SCORE"].agg({'mean': np.mean, 'std_dev': np.std, 'count': len})
# add standard erro
gzips_agg['std_error'] = gzips_agg['std_dev']/np.sqrt(gzips_agg['count'])

In [43]:
# select only top 100 scores
final_gzips = gzips_agg.sort(columns = 'mean', ascending=False)
final_gzips = final_gzips[final_gzips['count'] > 100][["mean", "std_error", "count"]]

In [44]:
#write to answer 1
final_gzips.to_csv("answers/a1.csv", line_terminator="),(", 
                  quoting= csv.QUOTE_NONNUMERIC)

In [45]:
# QUESTION 3
##############################

# Return a list of tuples of the form
#  `(borough, mean grade, stderr, number of inspections)`
#  for each of the city's five boroughs.  
# **Hint**: you will have to perform a join with the `boroughs` table.  
# Sort the list in ascending order by score.


webextract.columns

Index([u'CAMIS', u'DBA', u'BORO', u'BUILDING', u'STREET', u'ZIPCODE', u'PHONE',
       u'CUISINECODE', u'INSPDATE', u'ACTION', u'VIOLCODE', u'SCORE',
       u'CURRENTGRADE', u'GRADEDATE', u'RECORDDATE'],
      dtype='object')

In [46]:
# add boro names and select out scores and boros

boro_names = pd.DataFrame({'boro_names' : pd.Series({1: "MANHATTAN", 2 : "THE BRONX", 3 : "BROOKLYN", 
                           4 : "QUEENS", 5 : "STATEN ISLAND"})} )
boro = webextract[['SCORE', 'BORO']].join(boro_names, on = ['BORO'])
boro = boro.drop('BORO', axis=1)
boro.head()

,SCORE,boro_names
0,2,THE BRONX
1,23,BROOKLYN
2,11,MANHATTAN
3,10,BROOKLYN
4,11,MANHATTAN


In [47]:
boro["boro_names"] = boro["boro_names"].astype(str) #change boro datatype to string

In [48]:
# group by boro
g_boro = boro.groupby('boro_names')

# do calculations per boro
boro_scores = g_boro['SCORE'].agg({'mean': np.mean, 'std_dev':np.std, 'count': len  })
boro_scores['std_error'] = boro_scores['std_dev']/np.sqrt(boro_scores['count'])
boro_scores = boro_scores[['mean', 'std_error', 'count']]

# sort by score
boro_score = boro_scores.sort('mean', ascending = False)

In [49]:
boro_score.to_csv('answers/a2.csv', line_terminator = '),(',quoting= csv.QUOTE_NONNUMERIC )

In [50]:
# # Return a list of the 75 tuples of the form
#   `(cuisine, mean grade, stderr, number of inspections)`
#   for each of the 75 cuisine types with at least 100 inspections. 
# **Hint**: you will have to perform a join with the `boroughs` table.  
# Sort the list in ascendig order by score.  

In [51]:
# get the cuisine data
cuisine = webextract[['CUISINECODE', 'SCORE']]

# clean up dataframe
cuisine = cuisine.dropna()
cuisine['SCORE'] = cuisine['SCORE'].astype(int)

In [52]:
# group by cuisine code
cuisine_g = cuisine.groupby('CUISINECODE')['SCORE']

# do the calcuations
cuisine_scores = cuisine_g.agg({'mean': np.mean, 'std_dev':np.std, 'count': len  })
cuisine_scores['std_error'] = cuisine_scores['std_dev']/np.sqrt(cuisine_scores['count'])

cuisine_scores = cuisine_scores[['mean', 'std_error', 'count']]

In [53]:
# get cuisine labels 
cuisine_labels = pd.io.parsers.read_csv("Cuisine.txt")
cuisine_labels.head(5)

,CUISINECODE,CODEDESC
0,2,African
1,3,American
2,5,Asian
3,15,Cajun
4,17,Caribbean


In [54]:
# join cuisine lables to cuisine scores
cuisine_final = pd.merge(cuisine_scores, cuisine_labels, 
                         left_index=True, right_on='CUISINECODE')
cuisine_final = cuisine_final.drop('CUISINECODE', axis=1)

# select top restaurants with more than 100 inspections
cuisine_final = cuisine_final[cuisine_final['count'] > 100]

cuisine_final = cuisine_final[['CODEDESC', 'mean', 'std_error', 'count']]

# sort by average score
cuisine_final = cuisine_final.sort('mean', ascending=False)




cuisine_final.head()

,CODEDESC,mean,std_error,count
42,Creole,30.145455,0.828039,660
39,Chinese/Cuban,27.424528,0.821818,424
69,Portuguese,26.685714,1.107902,245
0,African,26.658837,0.454865,1788
30,Bangladeshi,26.586650,0.654190,779


In [55]:
cuisine_final.to_csv("answers/a3.csv", 
                     line_terminator = '),(', quoting=csv.QUOTE_NONNUMERIC,
                    index=False)

In [56]:
# VIOLATIONS
######################
webextract.columns

Index([u'CAMIS', u'DBA', u'BORO', u'BUILDING', u'STREET', u'ZIPCODE', u'PHONE',
       u'CUISINECODE', u'INSPDATE', u'ACTION', u'VIOLCODE', u'SCORE',
       u'CURRENTGRADE', u'GRADEDATE', u'RECORDDATE'],
      dtype='object')

In [57]:
webextract.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462,7188924968,8,2014-03-03 00:00:00,D,10F,2,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000
1,30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225,7182875005,39,2014-07-01 00:00:00,F,06A,23,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,10B,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
3,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234,7184443838,27,2014-05-29 00:00:00,D,08C,10,A,2014-05-29 00:00:00,2014-09-04 06:01:28.403000000
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019,2122452912,3,2013-07-22 00:00:00,D,02G,11,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000


In [58]:
# violations per restaurant inspection

violate = webextract[['VIOLCODE', 'CUISINECODE', 'INSPDATE']]
violate = violate.dropna()
violate = violate.reset_index(drop=True)


# need dates
from datetime import datetime
violate['INSPDATE'] = pd.to_datetime(violate['INSPDATE'])

violate.shape

(520720, 3)

# Cleaning violation

In [59]:
# # first clean up

# # clean up violation
import re

# # remove html tags
def remove_a(text):
    temp =  re.sub(r"<a.*A>", "", text)
    temp = re.sub(r"<a.*</a>", "", temp)
    return temp

# # read violation in CHEATED - removed degree symbols by hand
with open('Violation.txt') as f:
    viol = f.readlines()

# # save clean violation
with open('Violation_Clean.csv', 'w') as f:
    for row in viol:
        temp = remove_a(row)
        temp = re.sub(r'(",")', r';;;', temp)
        temp = re.sub(r'^"', '', temp)
        temp = re.sub(r'"\r\n', r'\r\n', temp)
        f.write(temp)

In [60]:
# write function to output 'iso-8859-1' 
def display_encode(head):
    f_answer = []
    for row in head.iterrows():
       f_answer.append((
            (row[1][0], row[1][1]), 
            row[1][2]))
    return f_answer

# with open('answers/violationratio.csv', 'wb') as f:
#         s = u"[" + y + u"]"
#         f.write(s)
#q = ', '.join(f_answer).decode('string_escape').decode('iso-8859-1')

# Violation labels in the data

In [61]:
violate_labels = pd.io.parsers.read_csv("Violation_Clean.csv", 
                                        sep=r';;;',
                                        encoding='iso-8859-1')

violate_labels = violate_labels.dropna()
violate_labels = violate_labels.reset_index(drop=False)
violate_labels['VIOLCODE'] = violate_labels['VIOLATIONCODE']
#violate_labels.drop('VIOLATIONCODE', axis=1, inplace=True)

/opt/conda/lib/python2.7/site-packages/pandas/io/parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  ParserWarning)


In [62]:
violate_labels.head()

,index,STARTDATE,ENDDATE,CRITICALFLAG,VIOLATIONCODE,VIOLATIONDESC,VIOLCODE
0,0,1901-01-01 00:00:00,2003-03-23 00:00:00,Y,01A,"Current valid , registration or other authori...",01A
1,1,2003-03-24 00:00:00,2005-02-17 00:00:00,Y,01A,"Current valid , registration or other authori...",01A
2,2,2005-02-18 00:00:00,2007-06-30 00:00:00,Y,01A,"Current valid , registration or other authori...",01A
3,3,2007-07-01 00:00:00,2008-06-30 00:00:00,Y,01A,"Current valid permit, registration or other au...",01A
4,4,2008-07-01 00:00:00,2009-08-01 00:00:00,Y,01A,"Current valid permit, registration or other au...",01A


In [63]:
# change to date time value
violate_labels['STARTDATE'] = pd.to_datetime(violate_labels['STARTDATE'])
violate_labels['ENDDATE'] = pd.to_datetime(violate_labels['ENDDATE'])

In [64]:
# merge the labels with the data
all_violate = pd.merge(violate_labels, violate)
all_violate = all_violate.drop(['index', 'CRITICALFLAG'],axis=1)
all_violate.head()

,STARTDATE,ENDDATE,VIOLATIONCODE,VIOLATIONDESC,VIOLCODE,CUISINECODE,INSPDATE
0,2005-02-18,2007-06-30,02A,Food not cooked to required minimum temperature.,02A,3,2014-07-07
1,2005-02-18,2007-06-30,02A,Food not cooked to required minimum temperature.,02A,48,2014-02-24
2,2005-02-18,2007-06-30,02A,Food not cooked to required minimum temperature.,02A,82,2014-03-24
3,2005-02-18,2007-06-30,02A,Food not cooked to required minimum temperature.,02A,20,2014-04-24
4,2005-02-18,2007-06-30,02A,Food not cooked to required minimum temperature.,02A,35,2014-08-28


In [65]:
# select out inspection date that lies between start and end date
between = (all_violate['INSPDATE'] > all_violate['STARTDATE']) & (all_violate['INSPDATE'] < all_violate['ENDDATE'])

In [66]:
f_violate = all_violate[between]

In [67]:
# now restrict to VIOLATIONDESC, CUISINECODE
f_violate = f_violate[['VIOLATIONDESC', 'CUISINECODE', 'VIOLCODE']].reset_index(drop=True)

In [68]:
f_violate.shape

(520720, 3)

In [81]:
# ON to probabilities.....

x =f_violate
x['count'] = 1
np.sum(x['count'])

520720

In [84]:
# The ratios don't mean much when the number of violations of a given type 
# and for a specific category are not large (why not?).  
#Be sure to filter these out.  We chose 100 as our cutoff.


# violations per cuisine per description code
group_viol_cuis = x.groupby(["CUISINECODE", "VIOLATIONDESC"])['count']
x['viol_per'] = group_viol_cuis.transform(np.sum)

# drop the count variable
#x = x.drop_duplicates()
#x = x.drop('count', axis=1)
#x = x.reset_index()

In [85]:
x.head()
np.sum(x['count'])

520720

In [86]:
# The ratios don't mean much when the number of violations of a given type 
# and for a specific category are not large (why not?).  
#Be sure to filter these out.  We chose 100 as our cutoff.

#x = x[x['viol_per'] > 100]

# have counts so we can clean up
#x = x.drop_duplicates()
#x = x.drop('count', axis=1)

#x = x.reset_index()
#np.sum(x['viol_per'])

In [87]:
# violations larger categories

#per cuisine violations

group_cuisine = x.groupby(["CUISINECODE"])['count']
x['per_cuisine'] = group_cuisine.transform(np.sum) 

In [88]:
#total violations per violation description
group_violation = x.groupby(["VIOLATIONDESC"])['count']
x['per_violation'] = group_violation.transform(np.sum) 

# total violations
x['total_violation'] = np.sum(x['count'])

x['total_violation'][0]

520720

In [89]:
# calculate conditional prob of violation per cusine code
x['prob_cuisine'] = x['viol_per']/ x['per_cuisine']

# calculat prob of violation
x['prob_viol'] = x['per_violation']/x['total_violation']

# compute ratios

x['ratio'] = x['prob_cuisine']/x['prob_viol']

In [90]:
x.sort('CUISINECODE', ascending=False).reset_index().loc[[0,1,2,3,4,5]]

,index,VIOLATIONDESC,CUISINECODE,VIOLCODE,count,viol_per,per_cuisine,per_violation,total_violation,prob_cuisine,prob_viol,ratio
0,517442,"Bulb not shielded or shatterproof, in areas wh...",99,22C,1,25,1881,4401,520720,0.013291,0.008452,1.572549
1,477258,Non-food contact surface improperly constructe...,99,10F,1,270,1881,66420,520720,0.143541,0.127554,1.125331
2,80017,Food Protection Certificate not held by superv...,99,04A,1,60,1881,9264,520720,0.031898,0.017791,1.792950
3,296556,Facility not vermin proof. Harborage or condit...,99,08A,1,137,1881,49561,520720,0.072834,0.095178,0.765237
4,484652,Proper sanitization not provided for utensil w...,99,10H,1,75,1881,8727,520720,0.039872,0.016759,2.379095
5,296541,Facility not vermin proof. Harborage or condit...,99,08A,1,137,1881,49561,520720,0.072834,0.095178,0.765237


In [91]:
# adjusting the form

# merge with cuisine_labels
answer = x[['VIOLATIONDESC', 'ratio', 'viol_per', 'CUISINECODE']].merge(cuisine_labels)

answer = answer.drop('CUISINECODE', axis=1)
answer = answer[['CODEDESC', 'VIOLATIONDESC', 'ratio', 'viol_per']]
answer = answer.sort('ratio', ascending=False)
answer = answer.reset_index(drop=True)

In [92]:
#write down the answer
f_answer = answer.drop_duplicates()


f_answer = f_answer[answer['viol_per'] > 100]
f_answer = f_answer[:20]

/opt/conda/lib/python2.7/site-packages/pandas/core/frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [93]:
f_answer = [str(((
            row[1][0], 
            row[1][1]
            ), 
            row[1][2], 
            row[1][3]) )  for row in f_answer.iterrows()]


# with open('answers/violationratio.csv', 'wb') as f:
#         s = u"[" + y + u"]"
#         f.write(s)
q = ', '.join(f_answer).decode('string_escape').decode('iso-8859-1')

In [94]:
f = open('answers/violationratio.csv', 'w')
f.write(q.encode('iso-8859-1'))
f.close()

In [95]:
print q

(('Japanese', 'Food worker does not use proper utensil to eliminate bare hand contact with food that will not receive adequate additional heat treatment.'), 3.244173663303972, 541), (('Café/Coffee/Tea', 'Choking first aid poster not posted. Alcohol and pregnancy warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted.'), 3.1528542316115793, 175), (('Juice, Smoothies, Fruit Salads', 'Food Protection Certificate not held by supervisor of food operations.'), 3.0895762870547507, 145), (('Donuts', 'Accurate thermometer not provided in refrigerated or hot holding equipment.'), 3.0373024983434593, 130), (('Ice Cream, Gelato, Yogurt, Ices', 'Food Protection Certificate not held by supervisor of food operations.'), 2.955949137148047, 193), (('Thai', 'Thawing procedures improper.'), 2.6329943302133367, 151), (('Irish', 'Raw, cooked or prepared food is adulterated, contaminated, cro